In [1]:
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import os
import numpy as np
import uuid

app = Flask(__name__)
Expected = {
    "cylinders":{"min":3, "max":8},
    "displacement":{"min":68.0, "max":455.0},
    "horsepower":{"min":46.0, "max":230.0},
    "weight":{"min":1613, "max":5140},
    "acceleration":{"min":8.0, "max":24.8},
    "year":{"min":70, "max":82},
    "origin":{"min":1, "max":3}
}

In [2]:
os.chdir(r'D:\Downloads\Python Practice\Deploying-Machine-Learning-with-Flask\saved-model')
model = load_model(os.path.join(os.getcwd(), 'mpg_model.h5'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 25)             │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 473 (1.85 KB)

 Trainable params: 471 (1.84 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
@app.route("/api", methods=['POST'])
def mpg_prediction():
    content = request.json
    errors = []
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f'Out Of Bounds:: {name} has value of {value}. Should fall between {expected_min} and {expected_max}')
        else:
            errors.append(f'Unexpected Field:: {name}')
    for name in Expected:
        if name not in content:
            errors.append(f'Missing Value:: {name}')
    if len(errors)<1:
        X = np.zeros((1,7))
        X[0,0] = content['cylinders']
        X[0,1] = content['displacement']
        X[0,2] = content['horsepower']
        X[0,3] = content['weight']
        X[0,4] = content['acceleration']
        X[0,5] = content['year']
        X[0,6] = content['origin']

        prediction = model.predict(X)
        mpg = float(prediction[0])
        response = {
            'id':str(uuid.uuid4()),
            'mpg':mpg,
            'errors': errors
            }
    else:
        response = {
            'id':str(uuid.uuid4()),
            'errors': errors
            }
        
    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


C:\Users\Sean\AppData\Local\Temp\ipykernel_1036\3406822179.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mpg = float(prediction[0])
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:18:35] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:19:39] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:19:56] "POST /api HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:21:40] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:21:42] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:21:44] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:22:53] "POST /api HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:27:28] "POST /api HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


INFO:werkzeug:127.0.0.1 - - [13/Aug/2024 10:30:33] "POST /api HTTP/1.1" 200 -
